### Parse XML and get metadata, paragraphs, sentences

In [58]:
deps = ["scripts/build_database.py"]

import sys
sys.path.append("/Users/jy006/Documents/Code/BioMedGPS/paper-parser")

from scripts.build_database import process_article

import xml.etree.ElementTree as ET

In [ ]:
xmlfile = "/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/extracted/oa_comm/oa_comm_xml.incr.2024-12-19/PMC006xxxxxx/PMC6627345
        
        
        
        .xml"

tree = ET.parse(xmlfile)
root = tree.getroot()
articles = root.findall('.//article')

metadata, paragraphs, sentences = process_article(root)

print(f"Found {len(metadata)} metadata, {len(paragraphs)} paragraphs, {len(sentences)} sentences")


In [ ]:
metadata

In [ ]:
paragraphs

In [ ]:
xmlfile = "/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/extracted/oa_comm/oa_comm_xml.incr.2024-12-25/PMC006xxxxxx/PMC6798053
        
        
        
        
        
        .xml"
tree = ET.parse(xmlfile)
root = tree.getroot()
articles = root.findall('.//article')

metadata, paragraphs, sentences = process_article(root)

print(f"Found {len(metadata)} metadata, {len(paragraphs)} paragraphs, {len(sentences)} sentences")


In [ ]:
metadata

In [ ]:
paragraphs

In [ ]:
import pandas as pd

# File path
file_path = "/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/processed/oa_comm/metadata/bb/bbbc3b8cc66a2adf1e460ef57ce1f5f2_20250103_152209.parquet"

# Attempt to read the Parquet file and display the first few rows
try:
    df = pd.read_parquet(file_path)
    print(df.head())
except Exception as e:
    print(f"An error occurred: {e}")

In [2]:
import os
import json 

data = json.load(open("/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/papers.json"))

pmids = [d.get("pmid") for d in data]

for pmid in pmids:
    if not os.path.exists(os.path.join("/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody", "pdfs", f"{pmid}.pdf")):
        print(f"Cannot find {pmid}")

### Count the number of paragraphs

In [ ]:
import duckdb

conn = duckdb.connect()
data = conn.sql(
    "SELECT COUNT(*) FROM read_parquet('/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/text_chunks/paragraphs.parquet')"
)

data.show()

### Count the number of records in the rocksdb db

In [ ]:
import os
import rocksdb

# 指定数据库路径
# root_dir = "/Users/jy006/Documents/Code/BioMedGPS/paper-parser/"
# db_path = os.path.join(
#     root_dir, "benchmarks", "antibody", "pmc_embedding", "test_rocksdb_no_speedup"
# )

db_path = "/work/data/projects/data2report/minillm-rocksdb-2025012500"

db = rocksdb.DB(
    db_path,
    rocksdb.Options(create_if_missing=False),
)

# 初始化计数器
num_records = 0
first_ten_records = []

# 创建迭代器
it = db.iterkeys()
it.seek_to_first()

# 遍历数据库
for key in it:
    num_records += 1
    # 记录前10条数据
    if num_records <= 10:
        value = db.get(key)
        first_ten_records.append((key, value))

# 输出结果
print(f"Number of records in the RocksDB database: {num_records}")
print("First 10 records:")
for i, (key, value) in enumerate(first_ten_records, 1):
    print(f"{i}: Key = {key}, Value = {value}")

### Deduplicate the embedding database

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointIdsList

client = QdrantClient(path="/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/pmc_embedding/minillm-vector-store")

# 获取所有数据
all_points = []
next_page = None

while True:
    response = client.scroll(
        collection_name="network-medicine",
        limit=1000,
        offset=next_page,
        with_payload=True,
        with_vectors=True,
    )
    all_points.extend(response[0])
    next_page = response[1]
    if next_page is None:
        break

# 去重逻辑
unique_points = {}
for point in all_points:
    text = point.payload.get("text")
    if text not in unique_points:
        unique_points[text] = point

# 获取去重后的数据
unique_points_list = list(unique_points.values())

# 可选：删除重复数据
unique_ids = {point.id for point in unique_points_list}
duplicate_ids = [point.id for point in all_points if point.id not in unique_ids]
if duplicate_ids:
    # 注意，这里要使用 PointIdsList，而不是直接用 dict
    client.delete(
        collection_name="network-medicine",
        points_selector=PointIdsList(points=duplicate_ids),
    )

# 去重后的数据重新存储
for point in unique_points_list:
    client.upsert(
        collection_name="network-medicine",
        points=[
            {
                "id": point.id,
                "vector": point.vector,
                "payload": point.payload,
            }
        ],
    )

print(f"去重完成，共保留 {len(unique_points_list)} 条记录")

### RocksDB to DuckDB

#### 单线程处理 RocksDB

In [ ]:
import duckdb
import rocksdb
import pandas as pd
import orjson
import os

# RocksDB 数据库路径
rocksdb_path = "/work/data/projects/data2report/minillm-rocksdb-2025012500"

# 存储 Parquet 目录
parquet_dir = "/work/data/projects/data2report/parquet_embeddings"
os.makedirs(parquet_dir, exist_ok=True)

# 连接 RocksDB
try:
    db = rocksdb.DB(rocksdb_path, rocksdb.Options(create_if_missing=False), read_only=True)
except Exception as e:
    print(f"无法打开 RocksDB: {e}")
    exit(1)

batch_size = 1_000_000  # 每 100 万条数据存一个 Parquet
batch = []
file_index = 0  # 记录当前 Parquet 文件编号

# 读取 RocksDB 迭代器
it = db.iterkeys()
it.seek_to_first()

processed_count = 0  # 记录已处理的数据量
log_interval = 100000  # 每 10 万条记录输出日志

for key in it:
    value = db.get(key)  # 获取 Value
    key_str = key.decode()  # 解码 Key
    
    try:
        # 解析 JSON，确保存储为 `LIST<FLOAT>` 而不是 `TEXT`
        embedding_list = orjson.loads(value)
        if isinstance(embedding_list, list) and all(isinstance(x, (float, int)) for x in embedding_list):
            batch.append((key_str, embedding_list))
    except json.JSONDecodeError:
        print(f"跳过无效数据: {key_str}")
        
    processed_count += 1  # 每处理 1 条数据，计数 +1

    # 每 10 万条打印一次日志
    if processed_count % log_interval == 0:
        print(f"🔹 已处理 {processed_count} 条数据")

    # 每 batch_size 条数据存一个 Parquet
    if len(batch) >= batch_size:
        parquet_file = os.path.join(parquet_dir, f"embeddings_{file_index}.parquet")
        df = pd.DataFrame(batch, columns=["key", "embedding"])
        df.to_parquet(parquet_file, compression="zstd", engine="pyarrow")
        print(f"✅ 已保存 {parquet_file}")
        batch = []
        file_index += 1

# 处理剩余数据
if batch:
    parquet_file = os.path.join(parquet_dir, f"embeddings_{file_index}.parquet")
    df = pd.DataFrame(batch, columns=["key", "embedding"])
    df.to_parquet(parquet_file, compression="zstd", engine="pyarrow")
    print(f"✅ 已保存 {parquet_file}")

print("🔹 所有数据已成功存为 Parquet！")


In [ ]:
import rocksdb
import duckdb
import orjson
import pandas as pd
import concurrent.futures
import time

def process_record(record):
    """
    处理单条记录：将 key 解码，将 JSON 格式的 value 解析成 Python 数值列表。
    """
    key, value = record
    try:
        # key 处理：如果是 bytes，则解码为字符串
        key_str = key.decode() if isinstance(key, bytes) else str(key)
        # value 处理：先解码，再用 json 解析
        json_text = value.decode() if isinstance(value, bytes) else str(value)
        vector = orjson.loads(json_text)
        if not isinstance(vector, list):
            raise ValueError("解析后的向量不是列表")
        # if len(vector) != 387:
        #     # 可记录警告，但不阻止后续处理
        #     print(f"警告：key {key_str} 的向量维度为 {len(vector)}，期望 387")
        return (key_str, vector)
    except Exception as e:
        print(f"处理 key {key} 时出错：{e}")
        return None

def migrate_rocksdb_to_duckdb(rocksdb_path, duckdb_path, batch_size=10000, num_workers=40):
    """
    将 RocksDB 中的数据批量迁移到 DuckDB 中，利用多进程并行解析 JSON，
    并对 DuckDB 做相应的性能优化。
    
    参数：
      rocksdb_path: RocksDB 数据库路径
      duckdb_path: 目标 DuckDB 数据库文件路径（文件不存在时会自动创建）
      batch_size: 每批次处理的记录数
      num_workers: 并行处理时的进程数（可根据 CPU 核心数调整）
    """
    # 设置 RocksDB 选项，并以只读方式打开数据库
    opts = rocksdb.Options()
    opts.create_if_missing = False
    db = rocksdb.DB(rocksdb_path, opts, read_only=True)
    
    # 连接 DuckDB，并设置高并发相关的 PRAGMA 参数
    con = duckdb.connect(duckdb_path)
    con.execute("PRAGMA threads=80;")               # 使用 80 个线程（充分利用 CPU）
    con.execute("PRAGMA memory_limit='120GB';")       # 提高内存限制，避免内存不足
    # 创建目标表，采用 key (VARCHAR) 和 vector (DOUBLE[]) 存储
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS vectors (
        key VARCHAR,
        vector DOUBLE[]
    );
    """
    con.execute(create_table_sql)
    
    # 获取 RocksDB 迭代器，顺序遍历所有记录
    it = db.iteritems()
    it.seek_to_first()
    
    batch_records = []   # 用于存储当前批次的 (key, value) 元组
    total_count = 0
    start_time = time.time()
    
    # 使用 ProcessPoolExecutor 进行多进程 JSON 解析
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        def process_batch(records):
            # 并行解析当前批次记录
            results = list(executor.map(process_record, records))
            # 过滤掉解析失败的记录
            return [r for r in results if r is not None]
        
        print("开始数据迁移...")
        for key, value in it:
            batch_records.append((key, value))
            total_count += 1
            if total_count % batch_size == 0:
                processed = process_batch(batch_records)
                if processed:
                    # 利用 Pandas DataFrame 组织批次数据
                    df = pd.DataFrame(processed, columns=["key", "vector"])
                    # 利用 DuckDB 的 register/INSERT 方式批量写入数据
                    con.register("tmp_df", df)
                    con.execute("INSERT INTO vectors SELECT * FROM tmp_df")
                    con.unregister("tmp_df")
                print(f"已处理并插入 {total_count} 条记录，耗时 {time.time()-start_time:.2f} 秒")
                batch_records = []  # 清空当前批次数据
        
        # 处理最后不足一个批次的数据
        if batch_records:
            processed = process_batch(batch_records)
            if processed:
                df = pd.DataFrame(processed, columns=["key", "vector"])
                con.register("tmp_df", df)
                con.execute("INSERT INTO vectors SELECT * FROM tmp_df")
                con.unregister("tmp_df")
            print(f"最终批次处理完毕，总计 {total_count} 条记录")
    
    con.close()
    print("数据迁移完成。")

if __name__ == "__main__":
    # 根据实际情况修改 RocksDB 数据库路径和目标 DuckDB 文件路径
    rocksdb_path = "/work/data/projects/data2report/minillm-rocksdb-2025012500"         # 例如："/data/rocksdb_db"
    duckdb_path = "/work/data/projects/data2report/paragraph-minillm-embedding-2025012500.duckdb"  # 例如："/data/target.duckdb"
    
    # 根据服务器配置调整 batch_size 和 num_workers 参数
    migrate_rocksdb_to_duckdb(rocksdb_path, duckdb_path, batch_size=10000, num_workers=40)


#### 多线程处理 RocksDB

In [ ]:
import duckdb
import rocksdb
import pandas as pd
import orjson
import os
import time
import threading
from queue import Queue
import pyarrow as pa
import pyarrow.parquet as pq

# RocksDB 数据库路径
rocksdb_path = "/work/data/projects/data2report/minillm-rocksdb-2025012500"

# 存储 Parquet 目录
parquet_dir = "/work/data/projects/data2report/paragraph-minillm-embedding-2025012500"
os.makedirs(parquet_dir, exist_ok=True)

# 连接 RocksDB
try:
    options = rocksdb.Options()
    
    options.create_if_missing = True
    options.error_if_exists = False
    options.paranoid_checks = True
    
    # MemTable 配置
    options.write_buffer_size = 1024 * 1024 * 1024 * 8
    options.max_write_buffer_number = 3
    options.min_write_buffer_number_to_merge = 3

    # Compaction 配置
    options.compression = rocksdb.CompressionType.snappy_compression

    # Block Cache 配置
    block_cache = rocksdb.LRUCache(1024 * 1024 * 1024 * 50)
    table_options = rocksdb.BlockBasedTableFactory(
        block_cache=block_cache,
    )
    options.table_factory = table_options

    # options.use_direct_reads = True
    # options.allow_mmap_reads = True
    # 后台线程配置
    options.max_background_flushes = 10
    options.max_background_compactions = 10
    db = rocksdb.DB(rocksdb_path, options, read_only=True)
except Exception as e:
    print(f"❌ 无法打开 RocksDB: {e}")
    exit(1)

# 参数设置
skip_top_k = 0    # 跳过前 0 条数据
batch_size = 1_000_000    # 每 100 万条数据做一次 multi_get
sub_batch_size = 100_000  # 每 10 万条数据交给一个线程处理（生成一个 Parquet 文件）
log_interval = 100_000    # 每 10 万条记录输出日志
thread_count = 10         # 工作线程数，可根据 CPU 资源调整
task_queue = Queue(maxsize=10)  # 限制任务队列长度，防止任务堆积

def process_batch(keys, values, file_idx):
    """
    处理 sub_batch_size 条数据并存为 Parquet。
    每个任务处理 10 万条数据（或不足 10 万条）。
    """
    batch = []
    start_time = time.time()

    decoded_values = [values.get(key).decode() for key in keys]
    decoded_keys = [key.decode() for key in keys]
    batch = [(key, orjson.loads(value)) for key, value in zip(decoded_keys, decoded_values)]

    # 假设 batch 是一个 [(key, embedding_list), ...] 的列表
    keys_list, embeddings_list = zip(*batch)  # 分离 keys 和 embeddings
    # 构建 pyarrow 数组
    key_array = pa.array(keys_list)
    embedding_array = pa.array(embeddings_list)  # 如果每个 embedding 长度一致，则可以构建二维数组；否则，构建 ListArray

    parquet_file = os.path.join(parquet_dir, f"embeddings_{file_idx}.parquet")
    table = pa.Table.from_arrays([key_array, embedding_array], names=['key', 'embedding'])
    pq.write_table(table, parquet_file, compression='snappy')
    print(f"✅ 已保存 {parquet_file}, 耗时 {time.time() - start_time:.2f} 秒")

def worker():
    """工作线程函数：不断从任务队列中取任务执行，遇到 None 时退出"""
    while True:
        task = task_queue.get()
        if task is None:
            task_queue.task_done()
            break
        sub_keys, sub_values, file_idx = task
        process_batch(sub_keys, sub_values, file_idx)
        task_queue.task_done()

# 启动工作线程
threads = []
for _ in range(thread_count):
    t = threading.Thread(target=worker, daemon=True)
    t.start()
    threads.append(t)

# 读取 RocksDB 数据
it = db.iterkeys()
it.seek_to_first()

keys = []
processed_count = 0
start_time = time.time()

while True:
    try:
        key = next(it)  # 获取下一个 key
        processed_count += 1

        # 在跳过阶段不累积 key
        if processed_count <= skip_top_k:
            continue

        keys.append(key)

        # 每 10 万条打印一次日志
        if processed_count % log_interval == 0:
            print(f"🔹 已读取 {processed_count} 条数据，耗时 {time.time() - start_time:.2f} 秒")

        if len(keys) >= batch_size:
            start_time_batch = time.time()
            values = db.multi_get(keys)  # 批量查询 RocksDB，减少 I/O
            print(f"⏳ multi_get 查询 {len(keys)} 条数据，耗时 {time.time() - start_time_batch:.2f} 秒")

            # 过滤掉 None 值，确保数据完整
            values = {k: v for k, v in values.items() if v is not None}

            batch_index = 0
            for i in range(0, batch_size, sub_batch_size):
                sub_keys = keys[i:i + sub_batch_size]
                sub_values = {k: values[k] for k in sub_keys if k in values}
                batch_index += 1
                task_queue.put((sub_keys, sub_values, f"{processed_count}_{batch_index}"))
                print(f"✅ 已提交 {len(sub_keys)} 条数据到线程池")
            keys = []  # 清空 keys 列表，准备下一批数据

    except StopIteration:
        # 遍历结束
        break

# 处理剩余数据（不足 batch_size 的部分）
if keys:
    start_time_sub = time.time()
    values = db.multi_get(keys)
    print(f"⏳ multi_get 查询 {len(keys)} 条数据，耗时 {time.time() - start_time_sub:.2f} 秒")
    for i in range(0, len(keys), sub_batch_size):
        sub_keys = keys[i:i + sub_batch_size]
        sub_values = {k: values[k] for k in sub_keys if k in values}
        task_queue.put((sub_keys, sub_values, processed_count + i))
        print(f"✅ 已提交 {len(sub_keys)} 条数据到线程池")

# 向每个工作线程发送终止信号
for _ in range(thread_count):
    task_queue.put(None)

# 等待所有任务完成
task_queue.join()

print("🔹 所有数据已成功提交到线程池处理！")

# 如果需要等待线程退出（虽然 daemon 线程在主线程退出时会被强行结束），可以 join：
for t in threads:
    t.join()


#### 合并 Parquet 文件到 DuckDB

In [ ]:
import re
import os
import duckdb

# Merge all parquet files into a duckdb database
parquet_dir = "/work/data/projects/data2report/paragraph-minillm-embedding-2025012500"

# 连接 DuckDB
duckdb_path = "/work/data/projects/data2report/paragraph-minillm-embedding-2025012500.duckdb"
con = duckdb.connect(duckdb_path)

con.execute("PRAGMA threads=80;")
con.execute("PRAGMA memory_limit='120GB';")

embedding_size = 384

# 创建 DuckDB 表
con.execute(f"""
    CREATE TABLE IF NOT EXISTS embeddings (
        key VARCHAR(32) UNIQUE,
        embedding FLOAT[{embedding_size}]
    );
""")

def extract_numbers(s):
    """提取文件名中的数字并转换为元组"""
    numbers = re.findall(r'\d+', s)  # 提取所有数字
    return tuple(map(int, numbers))  # 转换为整数元组进行排序

# 批量导入所有 Parquet
parquet_files = sorted([os.path.join(parquet_dir, f) for f in os.listdir(parquet_dir) if f.endswith(".parquet")], key=extract_numbers)

for parquet_file in parquet_files:
    con.execute(f"""
        INSERT INTO embeddings (key, embedding)
        SELECT * FROM read_parquet('{parquet_file}')
    """)
    print(f"✅ 已合并 {parquet_file}")

# **创建索引**
con.execute("CREATE UNIQUE INDEX idx_key ON embeddings(key);")

# 关闭数据库
con.close()

print("🎉 所有 Parquet 文件已合并到 DuckDB，并创建索引！")


In [ ]:
import os
import duckdb

root_dir = "/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk"
conn = duckdb.connect()

sql = f"""
PRAGMA threads=80; -- 启用多线程
PRAGMA memory_limit='120GB'; -- 限制内存占用

ATTACH '{root_dir}/paragraph-minillm-embedding-2025012500.duckdb' AS db;
CREATE TABLE db.embeddings AS 
SELECT * FROM read_parquet(['{os.path.join(root_dir, "paragraph-minillm-embedding-2025012500")}/*.parquet']);
"""

conn.sql(sql)

### Merge two vector databases

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# 初始化 Qdrant clients
target_client = QdrantClient(path="/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/pmc_embedding/nvembed-vector_db")
source_client = QdrantClient(path="/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/pmc_embedding/benchmark_datasets-vector_db")

# 合并数据
offset = None  # 初始化 offset
is_end = False
while True:
    # 批量获取数据
    points, offset = source_client.scroll(
        collection_name="network-medicine",
        limit=100,          # 每次获取 100 条
        offset=offset,      # 上次返回的偏移量
        with_payload=True,  # 包含 payload
        with_vectors=True,  # 包含向量
    )
    
    if len(points) < 100:
        is_end = True
    
    if not points:
        print("No more points to scroll. Merging completed.")
        break

    print(f"Found {len(points)} points.")

    # 插入到目标 collection
    target_client.upsert(
        collection_name="network-medicine",
        points=points,
    )
    
    if is_end:
        break

print("Data merge completed successfully.")


In [ ]:
print(source_client.count(collection_name="network-medicine"))
print(target_client.count(collection_name="network-medicine"))

### List parquet metadata

In [ ]:
import duckdb

# 读取 Parquet 文件元数据
df = duckdb.query(
    "SELECT * FROM parquet_metadata('/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/paragraphs_md5sum/paragraphs_0.parquet')"
).df()

# 显示元数据
print(df)

### Compute md5sums of all texts in a parquet file

In [ ]:
import duckdb
import hashlib
import polars as pl

# 定义批处理大小
batch_size = 1000000  # 每批次处理100万行

conn = duckdb.connect()

def count(conn, parquet_file):
    sql = f"SELECT COUNT(*) FROM read_parquet('{parquet_file}')"
    return conn.sql(sql).fetchone()[0]


# 获取数据总行数，以便按批次处理
row_count = count(conn, "/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/paragraphs.parquet")

print("Found", row_count, "paragraphs")

for start in range(0, row_count, batch_size):
    print(f"Processing batch {start} to {start + batch_size}")

    # 读取当前批次的数据
    batch = pl.scan_parquet(f"/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/paragraphs.parquet").slice(start, batch_size).collect()

    batch = batch.with_columns(
        pl.col("text")
        .map_elements(
            lambda x: hashlib.md5(x.encode("utf-8")).hexdigest(),
            return_dtype=pl.Utf8,
        )
        .alias("md5sum")
    )

    print(f"Writing batch {start} to {start + batch_size}")
    batch.write_parquet(
        f"/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/paragraphs_md5sum/paragraphs_{start}.parquet",
        row_group_size=100000,
        compression="zstd",
        compression_level=3,
        statistics=True,
        use_pyarrow=True,
    )

### Get all points from a qdrant collection

In [ ]:
import pandas as pd

# 分批加载所有点（假设点数较多）
all_points = []
batch_size = 1000
offset = 0

while True:
    points = neural_searcher.qdrant_client.scroll(
        collection_name="network-medicine",
        limit=batch_size,
        offset=offset,
        with_payload=True,
    )
    all_points.extend(points[0])
    if len(points[0]) < batch_size:  # 如果已经加载完所有数据
        break
    offset += batch_size

# 提取 payload 数据
payloads = [point.payload for point in all_points]

# 转为 DataFrame
df = pd.DataFrame(payloads)

# 按 pmid 分组统计
pmid_counts = df.groupby("pmid").size()
unique_pmids = pmid_counts.count()
print(f"Unique PMIDs: {unique_pmids}")

### Using an HNSW Index

In [ ]:
import duckdb

# List versions
duckdb.__version__

In [ ]:
# List all extensions
conn = duckdb.connect()
conn.sql("SELECT * FROM duckdb_extensions();").df()

In [ ]:
import os
root_dir = "/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/paragraphs_md5sum/"
parquet_files = os.listdir(root_dir)
parquet_files = [os.path.join(root_dir, f) for f in parquet_files if f.endswith(".parquet")]

# Create a new duckdb database
conn.sql("ATTACH '/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/paragraphs_md5sum.duckdb' AS db;");
conn.sql(f"CREATE TABLE db.embeddings AS SELECT * FROM read_parquet('{parquet_files[0]}');");

for parquet_file in parquet_files[1:]:
    print(f"Copying {parquet_file} to db.embeddings")
    conn.sql(f"COPY db.embeddings FROM '{parquet_file}' (FORMAT PARQUET)");

In [ ]:
# Install the hnsw extension
conn.sql("INSTALL vss;")
# Load the hnsw extension
conn.sql("LOAD vss;")

# Create a VSS index
conn.sql("CREATE INDEX vss_index ON embeddings USING HNSW (embedding);").df()


### Create VSS Index for an embedding column

In [ ]:
import os
import duckdb

root_dir = "/work/data/projects/data2report"
conn = duckdb.connect(
    os.path.join(root_dir, "paragraph-minillm-embedding-2025012500.duckdb")
)
vss_path = "/work/data/projects/data2report/vss.duckdb_extension"

print("Creating vss index")
# Install the hnsw extension
# conn.sql("INSTALL vss;")
# Load the hnsw extension
con.execute("PRAGMA threads=80;")
con.execute("PRAGMA memory_limit='120GB';")
conn.sql(f"LOAD '{vss_path}';")
conn.sql("PRAGMA hnsw_enable_experimental_persistence=true;")
conn.execute("CREATE INDEX vss_index ON embeddings USING HNSW (embedding);").fetchall()

### Create FAISS Index

https://medium.com/@deveshbajaj59/scaling-semantic-search-with-faiss-challenges-and-solutions-for-billion-scale-datasets-1cacb6f87f95

In [ ]:
import os
import duckdb
import faiss
import numpy as np
import math

# ------------------------
# 配置参数
# ------------------------
root_dir = "/work/data/projects/data2report"
DB_PATH = os.path.join(root_dir, "paragraph-minillm-embedding-2025012500.duckdb")
TABLE_NAME = "embeddings"
VECTOR_DIM = 384  # 假设 embedding 的维度为 384
INDEX_FILE_DIR = os.path.join(root_dir, "paragraph-minillm-embedding-2025012500")  # FAISS 索引存储文件

# ------------------------
# 连接 DuckDB 并获取总数据量
# ------------------------
conn = duckdb.connect(DB_PATH)
total_rows = conn.execute(f"SELECT COUNT(*) FROM {TABLE_NAME}").fetchone()[0]
print(f"数据库中共有 {total_rows} 条 embedding 记录")

# ------------------------
# 参数设置：分片数、每个分片的记录数、每个批次加载的记录数
# ------------------------
num_shards = 5
# 采用向上取整，保证所有记录都能被处理
num_records_per_shard = math.ceil(total_rows / num_shards)
# 如果每个分片的记录数过大，可以进一步分批处理，每个批次加载的记录数独立设定
BATCH_SIZE = 100000  # 根据内存和性能要求调整

print(f"每个分片大约处理 {num_records_per_shard} 条记录，每个批次加载 {BATCH_SIZE} 条记录，共 {num_shards} 个分片")

keys_list = []   # 存储 key 列表
total_vectors = 0  # 统计已加载的向量数量

try:
    cursor = conn.cursor()
    # 按分片处理数据
    for shard in range(num_shards):
        shard_start = shard * num_records_per_shard
        # 最后一个分片可能不足 num_records_per_shard
        shard_end = min(total_rows, (shard + 1) * num_records_per_shard)
        print(f"\n正在处理分片 {shard+1}/{num_shards}，记录范围：{shard_start} ~ {shard_end}")
        
        # ------------------------
        # 初始化 FAISS 索引（示例中采用 IVFPQ 索引，先在 CPU 上创建，再迁移到所有 GPU）
        # ------------------------
        quantizer = faiss.IndexFlatL2(VECTOR_DIM)
        faiss_index = faiss.IndexIVFPQ(quantizer, VECTOR_DIM, 1024, 32, 8)
        gpu_faiss_index = faiss.index_cpu_to_all_gpus(faiss_index)

        # 在当前分片内再按批次加载数据
        for offset in range(shard_start, shard_end, BATCH_SIZE):
            batch_end = min(offset + BATCH_SIZE, shard_end)
            current_batch_size = batch_end - offset
            print(f"  正在加载数据 {offset} - {batch_end} （批次大小：{current_batch_size}）")

            query = f"SELECT key, embedding FROM {TABLE_NAME} LIMIT {current_batch_size} OFFSET {offset}"
            cursor.execute(query)

            batch_keys = []
            batch_vectors = []
            # 逐步获取数据，避免一次性加载太多数据到内存
            for row in cursor.fetchmany(current_batch_size):
                batch_keys.append(row[0])
                # row[1] 应该为 embedding 向量（以数组或列表形式存储），转换为 np.float32 数组
                batch_vectors.append(np.array(row[1], dtype=np.float32))

            if batch_vectors:
                batch_vectors_np = np.vstack(batch_vectors)
                # 注意：FAISS 索引通常只需要训练一次
                if not gpu_faiss_index.is_trained:
                    print("  正在训练 FAISS 索引...")
                    gpu_faiss_index.train(batch_vectors_np)
                print("  添加 batch 到 FAISS 索引...")
                gpu_faiss_index.add(batch_vectors_np)
                keys_list.extend(batch_keys)
                total_vectors += len(batch_keys)
                print(f"  当前分片已加载 {len(batch_keys)} 条记录，总共加载 {total_vectors} 条记录")
            else:
                print("  当前批次没有数据")
    
        print(f"\n总共加载 {total_vectors} 条 embedding 进入 FAISS 索引")
        # 将索引从 GPU 迁回 CPU，并保存到磁盘
        output_filepath = os.path.join(INDEX_FILE_DIR, f"faiss_index_{shard}.faiss")
        faiss.write_index(faiss.index_gpu_to_cpu(gpu_faiss_index), output_filepath)
        print(f"FAISS 索引已保存到 {output_filepath}")
        
        # Save the keys list to a file
        keys_filepath = os.path.join(INDEX_FILE_DIR, f"keys_list_{shard}.json")
        with open(keys_filepath, "w") as f:
            json.dump(keys_list, f)
        print(f"Keys list已保存到 {keys_filepath}")
        
        keys_list = []
        total_vectors = 0
        batch_keys = []
        batch_vectors = []
        del gpu_faiss_index
        del faiss_index
        del quantizer

except Exception as e:
    print(f"发生错误: {e}")

finally:
    conn.close()


#### Parquet Files + FAISS

In [ ]:
import os
import duckdb
import faiss
import numpy as np
import polars as pl
import time
import json
import gc
import re
from collections import OrderedDict
from concurrent.futures import ThreadPoolExecutor, as_completed

# ------------------------
# 配置参数
# ------------------------

def extract_numbers(s):
    """提取文件名中的数字并转换为元组"""
    numbers = re.findall(r'\d+', s)  # 提取所有数字
    return tuple(map(int, numbers))  # 转换为整数元组进行排序

root_dir = "/work/data/projects/data2report"
parquet_dir = os.path.join(root_dir, "paragraph-minillm-embedding-2025012500")
# 批量导入所有 Parquet
parquet_files = sorted([os.path.join(parquet_dir, f) for f in os.listdir(parquet_dir) if f.endswith(".parquet")], key=extract_numbers)

TABLE_NAME = "embeddings"
VECTOR_DIM = 384  # 假设 embedding 的维度为 384
INDEX_FILE_DIR = os.path.join(root_dir, "paragraph-faiss-2025012500")  # FAISS 索引和 key 文件存储目录

if not os.path.exists(INDEX_FILE_DIR):
    os.makedirs(INDEX_FILE_DIR, exist_ok=True)

num_gpus = 3                     # 可用 GPU 数量
num_shards = 9

def group_files_by_prefix(parquet_files: list[str]) -> OrderedDict:
    """
    将文件列表按照文件名中的组标识（如数字部分）进行分组。
    假设文件名格式为 "embeddings_<group>_something.parquet"，
    返回一个有序字典，其键为组标识，值为该组的文件列表（排序后）。
    """
    pattern = re.compile(r'embeddings_(\d+)')
    groups = {}
    for f in parquet_files:
        basename = os.path.basename(f)
        m = pattern.search(basename)
        if m:
            group = m.group(1)
        else:
            group = basename  # 如果无法匹配，则以完整文件名作为分组键
        groups.setdefault(group, []).append(f)
    
    # 对每个组内部进行排序（例如按文件名排序）
    for group in groups:
        groups[group].sort()
    
    # 将组按照数值排序（组标识都是数字字符串）
    sorted_groups = OrderedDict(
        sorted(groups.items(), key=lambda x: int(x[0]))
    )
    return sorted_groups

def partition_by_groups(parquet_files: list[str], num_shards: int) -> list[list[str]]:
    """
    按照组将 parquet_files 分片，使得相同组的文件保持在一起。
    采用贪心算法累加组的文件数，直到达到目标数量，再开始下一个分片。
    """
    # 先按组分组
    grouped = group_files_by_prefix(parquet_files)
    
    # 计算总文件数
    total_files = len(parquet_files)
    # 目标每个分片的文件数（平均值）
    target = total_files / num_shards

    shards = []
    current_shard = []
    current_count = 0
    
    for group, files in grouped.items():
        # 如果当前分片中已有文件且加上当前组后会超过目标，
        # 且分片数量还未达到 num_shards-1（保证最后一片可以拿到剩余文件），
        # 则开始一个新的分片
        if current_shard and current_count + len(files) > target and len(shards) < num_shards - 1:
            shards.append(current_shard)
            current_shard = []
            current_count = 0
        current_shard.extend(files)
        current_count += len(files)
    
    # 添加最后一个分片
    if current_shard:
        shards.append(current_shard)
    
    return shards

def process_shard(shard: int):
    """
    处理单个分片：
      - 计算分片在总数据中的起始与结束位置
      - 每个分片内分批加载数据
      - 在指定 GPU 上构建 FAISS 索引，并将索引和 key 列表保存到磁盘
    """
    local_parquet_files = partition_by_groups(parquet_files, num_shards)[shard]
    print(f"处理分片 {shard} 的文件: {len(local_parquet_files)} {local_parquet_files}")
    
    # return
    output_filepath = os.path.join(INDEX_FILE_DIR, f"faiss_index_{shard}.faiss")
    
    if os.path.exists(output_filepath):
        print(f"[Shard {shard}] FAISS 索引已存在，跳过处理")
        return
    
    # 每个线程独立建立 DuckDB 连接
    keys_list = []       # 用于保存该分片所有记录的 key
    embeddings_list = []
    total_vectors = 0    # 统计该分片已加载的向量数量

    # ------------------------
    # 初始化 FAISS 索引，并指定使用的 GPU（device_id = shard % num_gpus）
    # ------------------------
    quantizer = faiss.IndexFlatL2(VECTOR_DIM)
    faiss_index = faiss.IndexIVFPQ(quantizer, VECTOR_DIM, 1024, 32, 8)
    device_id = shard % num_gpus
    gpu_res = faiss.StandardGpuResources()
    gpu_faiss_index = faiss.index_cpu_to_gpu(gpu_res, device_id, faiss_index)
    
    # 按批次加载当前分片内的数据
    for file in local_parquet_files:
        start_time = time.time()
        conn = duckdb.connect()
        df = conn.execute(f"SELECT key, embedding FROM read_parquet('{file}')").df()
        batch_keys = df["key"].to_list()
        batch_vectors = np.array(df["embedding"].to_list(), dtype=np.float32)
        print(f"[Shard {shard}] 读取/转换数据 {file} 耗时: {time.time() - start_time} 秒")
        
        keys_list.extend(batch_keys)
        total_vectors += len(batch_keys)
        
        start_time = time.time()
        if len(batch_vectors) > 0 and len(batch_vectors) < 1024:
            embeddings_list.extend(batch_vectors)
            continue
        elif len(batch_vectors) > 1024:
            embeddings_list.extend(batch_vectors)
        else:
            print(f"[Shard {shard}] 当前批次没有数据")
            continue
            
        stacked_embeddings_list = np.vstack(embeddings_list)
        # FAISS 索引只需要训练一次，训练步骤放在第一次批次中
        if not gpu_faiss_index.is_trained:
            print(f"[Shard {shard}] 正在训练 FAISS 索引...")
            gpu_faiss_index.train(stacked_embeddings_list)

        print(f"[Shard {shard}] 添加 batch 到 FAISS 索引...")
        gpu_faiss_index.add(stacked_embeddings_list)
        print(f"[Shard {shard}] 当前分片总共加载 {total_vectors} 条记录")
        print(f"[Shard {shard}] 添加 batch 到 FAISS 索引 耗时: {time.time() - start_time} 秒")
        
        embeddings_list = []

    # 保存当前分片构建的 FAISS 索引
    faiss.write_index(faiss.index_gpu_to_cpu(gpu_faiss_index), output_filepath)
    print(f"[Shard {shard}] FAISS 索引已保存到 {output_filepath}")
    
    # 保存 key 列表到 JSON 文件
    keys_filepath = os.path.join(INDEX_FILE_DIR, f"keys_list_{shard}.json")
    with open(keys_filepath, "w") as f:
        json.dump(keys_list, f)
    print(f"[Shard {shard}] Keys list 已保存到 {keys_filepath}")
    
    conn.close()
    
    # 显式释放 GPU 相关对象，并调用垃圾回收
    del gpu_faiss_index, faiss_index, quantizer, gpu_res
    gc.collect()


# ------------------------
# 利用多线程并行处理各个分片
# ------------------------
with ThreadPoolExecutor(max_workers=num_gpus) as executor:
    # 为每个分片提交一个任务
    futures = [executor.submit(process_shard, shard) for shard in range(num_shards)]
    # 等待所有任务完成
    for future in as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"处理分片时发生错误: {e}")

print("所有分片均已处理完毕！")


#### Duckdb + FAISS

In [ ]:
import os
import duckdb
import faiss
import numpy as np
import math
import time
import json
import gc
from concurrent.futures import ThreadPoolExecutor, as_completed

# ------------------------
# 配置参数
# ------------------------
root_dir = "/work/data/projects/data2report"
DB_PATH = os.path.join(root_dir, "paragraph-minillm-embedding-2025012500.duckdb")
TABLE_NAME = "embeddings"
VECTOR_DIM = 384  # 假设 embedding 的维度为 384
INDEX_FILE_DIR = os.path.join(root_dir, "paragraph-faiss-2025012500")  # FAISS 索引和 key 文件存储目录

if not os.path.exists(INDEX_FILE_DIR):
    os.makedirs(INDEX_FILE_DIR, exist_ok=True)

# 分片和批次参数
num_shards = 9                   # 总分片数（每个线程处理一个分片）
batch_size = 100000              # 每个批次加载的记录数（可根据内存和性能调整）
num_gpus = 3                     # 可用 GPU 数量

# ------------------------
# 获取总数据量（主线程中获取一次，然后传递给各线程）
# ------------------------
conn = duckdb.connect(DB_PATH, read_only=True)
total_rows = conn.execute(f"SELECT COUNT(*) FROM {TABLE_NAME}").fetchone()[0]
conn.close()
print(f"数据库中共有 {total_rows} 条 embedding 记录")

granularity = 5000000


def get_shard_ranges(total_rows: int, num_shards: int, granularity: int = 100000) -> list[tuple[int, int]]:
    """
    按照给定的粒度（默认 100,000 行）划分分片：
      - 计算平均值 avg = total_rows / num_shards，
      - 将 avg 按粒度向上取整得到 base，
      - 计算最大可分片数 max_shards = floor(total_rows / base) + (1 if total_rows % base != 0 else 0)；
      - 如果 max_shards 小于计划的 num_shards，则自动调整 num_shards 为 max_shards。
    返回一个包含每个分片 (start, end) 的列表，其中 end 为不包含边界。
    """
    avg = total_rows / num_shards
    base = int(math.ceil(avg / granularity)) * granularity

    # 计算按 base 能分成多少分片
    max_shards = math.floor(total_rows / base) + (1 if total_rows % base != 0 else 0)
    if max_shards < num_shards:
        print(f"注意：按当前粒度，每份 {base} 行，实际最多可分 {max_shards} 份；将自动调整分片数为 {max_shards}")
        num_shards = max_shards

    ranges = []
    start = 0
    # 生成前 num_shards - 1 个分片
    for i in range(num_shards - 1):
        end = start + base
        if end > total_rows:
            end = total_rows
        ranges.append((start, end))
        start = end
    # 最后一个分片取剩余部分（可能不足 base 行）
    if start < total_rows:
        ranges.append((start, total_rows))
    return ranges


def process_shard(shard: int):
    """
    处理单个分片：
      - 计算分片在总数据中的起始与结束位置
      - 每个分片内分批加载数据
      - 在指定 GPU 上构建 FAISS 索引，并将索引和 key 列表保存到磁盘
    """
    output_filepath = os.path.join(INDEX_FILE_DIR, f"faiss_index_{shard}.faiss")
    
    if os.path.exists(output_filepath):
        print(f"[Shard {shard}] FAISS 索引已存在，跳过处理")
        return
    
    # 每个线程独立建立 DuckDB 连接
    conn = duckdb.connect(DB_PATH, read_only=True)
    conn.execute("PRAGMA threads=80;")
    conn.execute("PRAGMA memory_limit='120GB';")
    shard_ranges = get_shard_ranges(total_rows, num_shards, granularity)
    print("Shard ranges:", shard_ranges)
    shard_start, shard_end = shard_ranges[shard]
    # shard_start = shard * num_records_per_shard
    # shard_end = min(total_rows, (shard + 1) * num_records_per_shard)
    print(f"[Shard {shard}] 处理记录范围：{shard_start} ~ {shard_end}")

    keys_list = []       # 用于保存该分片所有记录的 key
    total_vectors = 0    # 统计该分片已加载的向量数量

    # ------------------------
    # 初始化 FAISS 索引，并指定使用的 GPU（device_id = shard % num_gpus）
    # ------------------------
    quantizer = faiss.IndexFlatL2(VECTOR_DIM)
    faiss_index = faiss.IndexIVFPQ(quantizer, VECTOR_DIM, 1024, 32, 8)
    # device_id = shard % num_gpus
    # gpu_res = faiss.StandardGpuResources()
    # gpu_faiss_index = faiss.index_cpu_to_gpu(gpu_res, device_id, faiss_index)
    gpu_faiss_index = faiss.index_cpu_to_all_gpus(faiss_index)
    
    cursor = conn.cursor()
    # 按批次加载当前分片内的数据
    for offset in range(shard_start, shard_end, batch_size):
        batch_end = min(offset + batch_size, shard_end)
        current_batch_size = batch_end - offset

        start_time = time.time()
        print(f"[Shard {shard}] 正在加载数据 {offset} - {batch_end} （批次大小：{current_batch_size}）")        
        query = f"SELECT key, embedding FROM {TABLE_NAME} LIMIT {current_batch_size} OFFSET {offset}"
        # query = f"""
        # SELECT key, embedding, row_number() OVER () AS rowid
        # FROM {TABLE_NAME}
        # WHERE rowid > {offset} AND rowid <= {batch_end}
        # """
        cursor.execute(query)
        print(f"[Shard {shard}] 正在加载数据 {offset} - {batch_end} 耗时: {time.time() - start_time} 秒")
        
        batch_keys = []
        batch_vectors = []
        # 使用 fetchmany 分批取数据，防止一次性加载过多数据
        for row in cursor.fetchmany(current_batch_size):
            batch_keys.append(row[0])
            # 假定 row[1] 存储的是 embedding 向量（数组或列表形式）
            batch_vectors.append(np.array(row[1], dtype=np.float32))
        
        if batch_vectors:
            batch_vectors_np = np.vstack(batch_vectors)
            # FAISS 索引只需要训练一次，训练步骤放在第一次批次中
            if not gpu_faiss_index.is_trained:
                print(f"[Shard {shard}] 正在训练 FAISS 索引...")
                gpu_faiss_index.train(batch_vectors_np)
            print(f"[Shard {shard}] 添加 batch 到 FAISS 索引...")
            gpu_faiss_index.add(batch_vectors_np)
            keys_list.extend(batch_keys)
            total_vectors += len(batch_keys)
            print(f"[Shard {shard}] 当前分片已加载 {len(batch_keys)} 条记录，总共加载 {total_vectors} 条记录")
        else:
            print(f"[Shard {shard}] 当前批次没有数据")
    
    # 保存当前分片构建的 FAISS 索引
    faiss.write_index(faiss.index_gpu_to_cpu(gpu_faiss_index), output_filepath)
    print(f"[Shard {shard}] FAISS 索引已保存到 {output_filepath}")
    
    # 保存 key 列表到 JSON 文件
    keys_filepath = os.path.join(INDEX_FILE_DIR, f"keys_list_{shard}.json")
    with open(keys_filepath, "w") as f:
        json.dump(keys_list, f)
    print(f"[Shard {shard}] Keys list 已保存到 {keys_filepath}")
    
    conn.close()
    
    # 显式释放 GPU 相关对象，并调用垃圾回收
    del gpu_faiss_index, faiss_index, quantizer, gpu_res
    gc.collect()


# ------------------------
# 利用多线程并行处理各个分片
# ------------------------
with ThreadPoolExecutor(max_workers=1) as executor:
    # 为每个分片提交一个任务
    futures = [executor.submit(process_shard, shard) for shard in range(num_shards)]
    # 等待所有任务完成
    for future in as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"处理分片时发生错误: {e}")

print("所有分片均已处理完毕！")


### Search by faiss

In [2]:
import os
import faiss
import numpy as np
import json

os.environ["HF_HOME"] = "/work/data/environments/cache/"

from sentence_transformers import SentenceTransformer

class NeuralSearcher:
    def __init__(self, faiss_dir: str, model_name="sentence-transformers/all-MiniLM-L6-v2", device=None):
        # Initialize encoder model
        self.model = SentenceTransformer(model_name, device=device if device is not None else "cpu", trust_remote_code=True)
        self.faiss_dir = faiss_dir
        
        self.index_files = sorted([os.path.join(faiss_dir, f) for f in os.listdir(faiss_dir) if f.endswith(".faiss")], key=lambda x: int(x.split("_")[-1].split(".")[0]))
        self.keys_list = sorted([os.path.join(faiss_dir, f) for f in os.listdir(faiss_dir) if f.endswith(".json")], key=lambda x: int(x.split("_")[-1].split(".")[0]))

    def _search(self, query_vector: str, top_k: int = 10):
        final_results = []
        for keys_file, index_file in zip(self.keys_list, self.index_files):
            faiss_index = faiss.read_index(index_file)
            res = faiss.StandardGpuResources()
            gpu_faiss_index = faiss.index_cpu_to_all_gpus(faiss_index)
            print(f"已加载 FAISS 索引，总条数：{faiss_index.ntotal}")
            
            keys_list = json.load(open(keys_file))

            def search_similar_vectors(query_vector, top_k=10):
                """
                在 FAISS 索引中查找最相似的 top_k 向量
                """
                query_vector = np.array(query_vector, dtype=np.float32).reshape(1, -1)  # 确保正确形状
                distances, indices = gpu_faiss_index.search(query_vector, top_k)  # 执行搜索

                # 解析查询结果
                nearest_keys = [keys_list[i] for i in indices[0]]
                distances = distances[0]
                
                return {
                    key: distance
                    for key, distance in zip(nearest_keys, distances)
                }


            top_results = search_similar_vectors(query_vector, top_k=top_k)

            final_results.append({
                "keys_file": keys_file,
                "index_file": index_file,
                "top_results": top_results
            })
        
        return final_results
        
    def search(self, text: str, limit: int = 10):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()

        # Use `vector` for search for closest vectors in the collection
        return self._search(vector, limit)

In [3]:
neural_searcher = NeuralSearcher(faiss_dir="/work/data/projects/data2report/paragraph-faiss-2025012500")
results1 = neural_searcher.search("How to express recombinant Interleukin-1 beta?", limit=10)

已加载 FAISS 索引，总条数：21000960
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：26250317


In [14]:
results2 = neural_searcher.search("How to express recombinant Interleukin-1 beta?", limit=100)

已加载 FAISS 索引，总条数：21000960
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：26250317


In [29]:
results3 = neural_searcher.search("How to express recombinant Interleukin-1 beta?", limit=1000)

已加载 FAISS 索引，总条数：21000960
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：21000000
已加载 FAISS 索引，总条数：26250317


In [5]:
import pandas as pd

key_scores = [result["top_results"] for result in results]
df = pd.DataFrame([
    {"key": k, "score": v}
    for d in key_scores
    for k, v in d.items()
])
df.sort_values(by="score", ascending=False, inplace=True)
keys = df["key"].tolist()

In [8]:
tests = {
    "PD-1": {
        "prompt": "How to express recombinant PD-1?",
        "dois": [
            "10.1080/19420862.2019.1596514",
            "10.1158/2159-8290.CD-20-1445",
            "10.1158/1535-7163.MCT-16-0665",
            "10.1080/19420862.2017.1371386",
            "10.1056/NEJMoa1200690",
            "10.1093/intimm/dxu095",
            "10.1016/j.critrevonc.2016.02.001",
            "10.1016/j.molmed.2014.10.009",
            "10.1080/14653240802320237",
            "10.1038/s41598-022-16776-1"
        ]
    },
    "TNF-alpha": {
        "prompt": "How to express recombinant TNF-alpha?",
        "dois": [
            "10.1016/j.intimp.2021.107418",
            "10.1182/blood-2011-04-325225",
            "10.1093/ecco-jcc/jjw053",
            "10.1080/21655979.2021.1967710",
            "10.1371/journal.pone.0006087",
            "10.1371/journal.pone.0016373",
            "10.3390/md20050307",
            "10.4049/jimmunol.2101180",
            "10.3390/jcm12041630",
            "10.1080/17425255.2017.1360280"
        ]
    },
    "Amyloid-beta": {
        "prompt": "How to express recombinant Amyloid-beta?",
        "dois": [
            "10.1016/j.biopsych.2017.08.010",
            "10.1038/aps.2017.28",
            "10.3233/JAD-179941",
            "10.1016/j.pep.2015.05.015",
            "10.1016/j.pep.2012.08.021",
            "10.3233/JAD-2011-110977",
            "10.1002/ana.24188",
            "10.1186/s13195-022-01117-1",
            "10.1186/alzrt246"
        ]
    },
    "Interleukin-1 beta": {
        "prompt": "How to express recombinant Interleukin-1 beta?",
        "dois": [
            "10.1210/en.2009-1124",
            "10.4161/mabs.28614",
            "10.1186/ar2438",
            "10.2337/dc12-1835",
            "10.1016/j.molimm.2013.08.002",
            "10.1080/19420862.2015.1081323",
            "10.4161/mabs.28614",
            "10.4161/mabs.3.1.13989"
        ]
    },
    "COVID-19 spike protein": {
        "prompt": "How to express recombinant COVID-19 spike protein?",
        "dois": [
            "10.1126/science.abc6952",
            "10.1016/j.vaccine.2014.04.016",
            "10.1038/s41591-020-0998-x",
            "10.1016/j.biopha.2020.110337",
            "10.1016/j.cell.2021.02.035",
            "10.1038/s41423-020-0426-7",
            "10.1080/21645515.2020.1740560",
            "10.1371/journal.pone.0300297",
            "10.1126/science.abc2241",
            "10.1126/science.abc5902"
        ]
    },
    "Helicobacter pylori": {
        "prompt": "How to express recombinant Helicobacter pylori?",
        "dois": [
            "10.1128/cdli.9.1.75-78.2002",
            "10.1128/IAI.70.8.4158-4164.2002",
            "10.1016/j.actbio.2022.02.031",
            "10.3390/microorganisms12061148",
            "10.1016/j.jim.2012.06.010",
            "10.1007/s00535-022-01933-0",
            "10.1007/s00253-018-9068-4",
            "10.1016/S0304-4165(00)00005-2",
            "10.1016/j.jim.2007.05.005"
        ]
    }
}


def load_papers(dois):
    with open("/work/data/projects/paper-parser/benchmarks/antibody/papers.json", "r") as f:
        papers = json.load(f)

    paper_metadata = [paper for paper in papers if paper["doi"] in dois]

    truth_pmids = [str(paper_metadata["pmid"]) for paper_metadata in paper_metadata]
    
    return truth_pmids

truth_pmids = load_papers(tests["Interleukin-1 beta"]["dois"])
truth_pmids

['26284424',
 '24671001',
 '24091292',
 '23514733',
 '21048425',
 '20332197',
 '18534016']

In [28]:
file = "/work/data/projects/data2report/paragraphs_md5sum/paragraphs_benchmark.parquet"
import duckdb

conn = duckdb.connect()

df = conn.execute(f"""
    SELECT md5sum, text, pmid, title
    FROM read_parquet('{file}')
    -- WHERE md5sum IN ({", ".join(map(repr, keys))})
""").df()

df

,md5sum,text,pmid,title
0,3123623c10be553a87bbbb9a62db4582,PD-L1 expression in cancer patients receiving ...,26895815,26895815-abstract
1,2b361300ff28b57c63143434a6e3f60b,Introduction\nA dynamic relationship exists be...,26895815,26895815-Introduction
2,c6ed448e85c0656e3f781c8eb56334c9,Meta-analysis methods\nA systematic literature...,26895815,26895815-Meta-analysis methods
3,12fc9a1c01dbd10d7852f8949710433b,Literature search\nPublished reports were obta...,26895815,26895815-Literature search
4,697856e680f1c2b606de23d708646aa5,Statistical analysis\nWe extracted data from a...,26895815,26895815-Statistical analysis
...,...,...,...,...
956,552e72f182c130a0d49b2583e00f7222,Fcγ-receptor mediated induction of wound heali...,26896086,26896086-Fcγ-receptor mediated induction of wo...
957,4c3f1796b2ae8d64aed4ec40ad4c6b67,Conclusions\nAnti-TNFs may have multiple modes...,26896086,26896086-Conclusions
958,dc5d9fbe4e9cd4913a358b292e7c2d3d,Funding\nNo specific funding was obtained for ...,26896086,26896086-Funding
959,023816cac915382a02118c89261a7c71,Conflict of Interest\nThe authors declare no c...,26896086,26896086-Conflict of Interest


In [20]:
import duckdb
import json

paragraph_md5sum_file = "/work/data/projects/data2report/paragraphs-md5sum-2025012501/md5sum_prefix=*/**.parquet"
conn = duckdb.connect()

conn.execute(f"""
PRAGMA threads=30;
PRAGMA memory_limit='120GB';
""")

# keys = json.load(open("/work/data/projects/data2report/keys.json"))
keys = list(results[0]["top_results"].keys())
md5sum_prefixes = [k[:2] for k in keys]

print("MD5SUM_PREFIXES: ", len(md5sum_prefixes), len(keys))

df = conn.execute(f"""
    SELECT md5sum, text, pmid, title
    FROM read_parquet('{paragraph_md5sum_file}')
    WHERE md5sum IN ({", ".join(map(repr, keys))}) AND md5sum_prefix IN ({", ".join(map(repr, md5sum_prefixes))})
""").df()

pmids = df["pmid"].tolist()

MD5SUM_PREFIXES:  10 10


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Partitioning by md5sum

In [ ]:
import duckdb
import os
import glob
from datetime import datetime

# 配置参数
INPUT_PARQUET = "/work/data/projects/data2report/paragraphs_md5sum/*.parquet"  # 原始数据路径（支持通配符）
OUTPUT_DIR = "/work/data/projects/data2report/paragraphs-md5sum-2025012501"       # 分区后数据输出目录
MIN_FILE_SIZE = "128MB"                    # 合并文件的最小大小

def setup_database():
    """配置 DuckDB 参数（优化性能）"""
    conn = duckdb.connect()
    conn.execute("PRAGMA threads=30;")              
    conn.execute("PRAGMA memory_limit='120GB';")     
    return conn

def init_bucketed_export(conn):
    """
    将原始数据按 md5sum 前两位分区导出。
    每个分区输出一个文件，文件名格式为 <min_md5>_<max_md5>_<num_rows>.parquet，
    其中 min_md5 和 max_md5 分别为该分区内 md5sum 的最小值和最大值，num_rows 为行数。
    """
    print("开始初始分桶导出...")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    # 获取所有不同的 md5sum 前缀（取前两位）
    prefixes = conn.execute(f"""
        SELECT DISTINCT substr(md5sum, 1, 2) as prefix 
        FROM read_parquet('{INPUT_PARQUET}')
    """).fetchall()
    
    for row in prefixes:
        prefix = row[0]
        # 查询该分区的最小和最大 md5sum
        min_max = conn.execute(f"""
            SELECT MIN(md5sum), MAX(md5sum) 
            FROM read_parquet('{INPUT_PARQUET}') 
            WHERE substr(md5sum, 1, 2) = '{prefix}'
        """).fetchone()
        min_md5, max_md5 = min_max
        # 计算该分区的行数
        num_rows = conn.execute(f"""
            SELECT COUNT(*) 
            FROM read_parquet('{INPUT_PARQUET}') 
            WHERE substr(md5sum, 1, 2) = '{prefix}'
        """).fetchone()[0]
        # 创建分区目录，以前缀命名
        partition_dir = os.path.join(OUTPUT_DIR, f"md5sum_prefix={prefix}")
        os.makedirs(partition_dir, exist_ok=True)
        # 构造输出文件路径，文件名格式: <min_md5>_<max_md5>_<num_rows>.parquet
        output_file = os.path.join(partition_dir, f"{min_md5}_{max_md5}_{num_rows}.parquet")
        sql = f"""
            COPY (
                SELECT * 
                FROM read_parquet('{INPUT_PARQUET}') 
                WHERE substr(md5sum, 1, 2) = '{prefix}'
            )
            TO '{output_file}'
            WITH (
                FORMAT PARQUET
            );
        """
        conn.execute(sql)
        print(f"分区 {prefix} 导出完成：{output_file}")
    print("所有分区分桶导出完成！")

def append_incremental_data(conn, new_data_parquet):
    """
    将增量数据按 md5sum 前两位追加到各分区。
    每个分区导出为单独的增量文件，文件名格式为 inc_<timestamp>.parquet。
    后续可调用 merge_small_files 进行合并。
    """
    print("开始增量数据追加...")
    prefixes = conn.execute(f"""
        SELECT DISTINCT substr(md5sum, 1, 2) as prefix 
        FROM read_parquet('{new_data_parquet}')
    """).fetchall()
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    for row in prefixes:
        prefix = row[0]
        partition_dir = os.path.join(OUTPUT_DIR, f"md5sum_prefix={prefix}")
        os.makedirs(partition_dir, exist_ok=True)
        output_file = os.path.join(partition_dir, f"inc_{timestamp}.parquet")
        sql = f"""
            COPY (
                SELECT * 
                FROM read_parquet('{new_data_parquet}') 
                WHERE substr(md5sum, 1, 2) = '{prefix}'
            )
            TO '{output_file}'
            WITH (
                FORMAT PARQUET
            );
        """
        conn.execute(sql)
        print(f"增量分区 {prefix} 导出完成：{output_file}")
    print("增量数据追加完成！")

def merge_small_files(conn):
    """
    合并各分区内的小文件。
    对于每个分区（目录名为 md5sum 的前两位），
    如果存在多个文件或单个文件小于阈值，则读取该分区所有数据，
    重新计算该分区的 md5sum 最小、最大值以及行数，并导出为新的文件，
    文件名格式为 <min_md5>_<max_md5>_<num_rows>.parquet，最后删除旧文件。
    """
    print("开始合并小文件...")
    # 遍历 OUTPUT_DIR 下所有目录（假定目录名为两位十六进制字符串）
    for prefix in os.listdir(OUTPUT_DIR):
        partition_dir = os.path.join(OUTPUT_DIR, f"md5sum_prefix={prefix}")
        if not os.path.isdir(partition_dir):
            continue
        files = glob.glob(os.path.join(partition_dir, "*.parquet"))
        if not files:
            continue
        # 如果只有一个文件且文件大小达到阈值，则无需合并
        if len(files) == 1:
            try:
                if os.path.getsize(files[0]) >= parse_file_size(MIN_FILE_SIZE):
                    continue
            except OSError as e:
                print(f"无法获取文件大小: {files[0]}, 错误: {e}")
        # 计算合并后的最小、最大 md5sum 和行数
        file_pattern = os.path.join(partition_dir, "*.parquet")
        result = conn.execute(f"""
            SELECT MIN(md5sum), MAX(md5sum), COUNT(*)
            FROM read_parquet('{file_pattern}')
        """).fetchone()
        min_md5, max_md5, num_rows = result
        merged_file = os.path.join(partition_dir, f"{min_md5}_{max_md5}_{num_rows}.parquet")
        merge_sql = f"""
            COPY (
                SELECT * FROM read_parquet('{file_pattern}')
            )
            TO '{merged_file}'
            WITH (
                FORMAT PARQUET, 
                FILE_SIZE '{MIN_FILE_SIZE}'
            );
        """
        conn.execute(merge_sql)
        print(f"分区 {prefix} 合并完成：{merged_file}")
        # 删除旧文件
        for f in files:
            try:
                os.remove(f)
            except OSError as e:
                print(f"删除文件失败: {f}, 错误: {e}")
    print("所有小文件合并完成！")

def parse_file_size(size_str):
    """将文件大小字符串（如 '128MB'）转换为字节数"""
    size_str = size_str.replace(" ", "")
    units = {"B": 1, "KB": 1024, "MB": 1024**2, "GB": 1024**3}
    num, unit = size_str[:-2], size_str[-2:]
    return int(num) * units[unit.upper()]

def query_example(conn, target_md5):
    """
    示例查询：根据目标 md5sum 提取前两位作为分区前缀，
    然后从对应分区下查询满足条件的数据。
    """
    try:
        prefix = target_md5[:2]
    except Exception as e:
        print(f"无效的 md5sum: {target_md5}, 错误: {e}")
        return []
    file_pattern = os.path.join(OUTPUT_DIR, f"md5sum_prefix={prefix}", "*.parquet")
    query = f"""
        SELECT * 
        FROM read_parquet('{file_pattern}')
        WHERE md5sum = '{target_md5}';
    """
    print(f"执行查询: {query}")
    return conn.execute(query).fetchall()

def main():
    conn = setup_database()
    
    # 初始分桶导出（全量数据）
    init_bucketed_export(conn)
    
    # 增量数据追加示例（模拟新数据）
    # append_incremental_data(conn, "/path/to/new_data.parquet")
    
    # 合并小文件（可选）
    # merge_small_files(conn)
    
    # 示例查询
    # result = query_example(conn, "d41d8cd98f00b204e9800998ecf8427e")
    # print(f"查询结果: {result}")
    
    conn.close()

main()
